In [ ]:
# os.system('echo $CUDA_HOME')
# os.environ['XLA_FLAGS'] = "--xla_gpu_cuda_data_dir=/usr/lib/nvidia-cuda-toolkit"
# os.environ['CUDA_HOME'] = "/usr/include/cuda/"
# os.environ['TF_GPU_ALLOCATOR'] = "cuda_malloc_async"
# os.system('echo $XLA_FLAGS')
# os.system('echo $CUDA_HOME')
# os.system('echo $TF_GPU_ALLOCATOR')

In [ ]:
import numpy as np
import PIL
import PIL.Image
import tensorflow as tf
import pathlib
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Reshape
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense

In [ ]:
print(tf.__version__)

In [ ]:
data_dir = pathlib.Path('./Datasets/lung/').with_suffix('')
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
bengin=list(data_dir.glob('Normal cases/*'))
PIL.Image.open(str(bengin[1]))

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=123,
  image_size=(512, 512),
  batch_size=32)

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(512, 512),
  batch_size=32)

In [ ]:
class_names_ = train_ds.class_names # type: ignore
print(class_names_)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1): # type: ignore
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names_[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
model=Sequential()
model.add(Conv2D(filters= 8,kernel_size=(4,4),input_shape=(512,512,3),padding='same'))
model.add(Conv2D(filters= 24,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters= 72,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=144,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=64,activation='relu'))
model.add(Dense(units=32,activation='relu'))
model.add(Dense(3))
model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping
es=EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=4,verbose=1,mode='auto')
mc=ModelCheckpoint(monitor="val_accuracy",filepath="./bestmodel.h5",verbose=1,save_best_only=True,mode='auto')
cd=[es,mc]

In [ ]:
import keras.backend as K
def recall(y_true, y_pred):
    y_true = K.ones_like(y_true)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    all_positives = K.sum(K.round(K.clip(y_true, 0, 1)))

    recall = true_positives / (all_positives + K.epsilon())
    return recall

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())
# import numpy as np
# import keras.backend as K

# def accuracy1(actual, predicted):
#     """accuracy = (tp + tn) / ts

#     , where:    

#         ts - Total Samples
#         tp - True Positives
#         tn - True Negatives
#     """
#     return (actual == predicted).mean() / float(len(actual))

In [ ]:

model.compile(
  optimizer='adam',
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  # loss='categorical_crossentropy',
  metrics=['accuracy'])

In [16]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10,
  use_multiprocessing=True
  # callbacks=cd
)

KeyboardInterrupt: 

In [ ]:
from keras.preprocessing.image import load_img,img_to_array
path="./Datasets/lung/Malignant cases/Malignant case (16).jpg"
img=load_img(path,target_size=(512,512))
arr=img_to_array(img)
arr=np.expand_dims(arr,axis=0)
pred=model.predict(arr)
print(pred)
pred_1=list(pred[0])
print(pred_1)
ans=list(class_names_)
print(ans)
print(ans[pred_1.index(max(pred_1))])